<a href="https://colab.research.google.com/github/mjmousavi97/Graph-Neural-Networks/blob/main/09_Graph_classification/src/graph_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.5 MB/s eta 0:00:00


In [13]:
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader

In [3]:
dataset = TUDataset(root='.', name='PROTEINS').shuffle()

In [4]:
print(f"dataset: {dataset}")
print(f"Number of Graphs: {len(dataset)}")
print(f"Number of Nodes: {dataset[0].num_nodes}")
print(f"Number of Features: {dataset.num_features}")
print(f"Number of Classes: {dataset.num_classes}")

dataset: PROTEINS(1113)
Number of Graphs: 1113
Number of Nodes: 91
Number of Features: 3
Number of Classes: 2


In [14]:
train_dataset = dataset[:int(len(dataset) * 0.8)]
val_dataset = dataset[int(len(dataset) * 0.8):int(len(dataset) * 0.9)]
test_dataset = dataset[int(len(dataset) * 0.9):]

print(f"Number of training graphs: {len(train_dataset)}")
print(f"Number of validation graphs: {len(val_dataset)}")
print(f"Number of test graphs: {len(test_dataset)}")

Number of training graphs: 890
Number of validation graphs: 111
Number of test graphs: 112


In [15]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print("\nTrain Loader:")
for i, batch in enumerate(train_loader):
    print(f" - batch{i+1}: {batch}")

print("\nValidation Loader:")
for i, batch in enumerate(val_loader):
    print(f" - batch{i+1}: {batch}")

print("\nTest Loader")
for i, batch in enumerate(test_loader):
    print(f" - batch{i+1}: {batch}")


Train Loader:
 - batch1: DataBatch(edge_index=[2, 9832], x=[2631, 3], y=[64], batch=[2631], ptr=[65])
 - batch2: DataBatch(edge_index=[2, 12702], x=[3532, 3], y=[64], batch=[3532], ptr=[65])
 - batch3: DataBatch(edge_index=[2, 7842], x=[2013, 3], y=[64], batch=[2013], ptr=[65])
 - batch4: DataBatch(edge_index=[2, 8314], x=[2224, 3], y=[64], batch=[2224], ptr=[65])
 - batch5: DataBatch(edge_index=[2, 10396], x=[2790, 3], y=[64], batch=[2790], ptr=[65])
 - batch6: DataBatch(edge_index=[2, 11188], x=[2918, 3], y=[64], batch=[2918], ptr=[65])
 - batch7: DataBatch(edge_index=[2, 8934], x=[2395, 3], y=[64], batch=[2395], ptr=[65])
 - batch8: DataBatch(edge_index=[2, 7480], x=[2012, 3], y=[64], batch=[2012], ptr=[65])
 - batch9: DataBatch(edge_index=[2, 8566], x=[2239, 3], y=[64], batch=[2239], ptr=[65])
 - batch10: DataBatch(edge_index=[2, 10290], x=[2716, 3], y=[64], batch=[2716], ptr=[65])
 - batch11: DataBatch(edge_index=[2, 8502], x=[2342, 3], y=[64], batch=[2342], ptr=[65])
 - batch12: